In [1]:
import pandas as pd
from sodapy import Socrata
import re
from sqlalchemy import create_engine
from config import USERNAME, APP_TOKEN, DB, PASSWORD

client = Socrata("data.lacity.org",
                  f'{APP_TOKEN}',
                  username=f"{USERNAME}",
                  password=f"{PASSWORD}")


results = client.get("2nrs-mtv8", order="date_rptd desc", limit=100000)
results_df = pd.DataFrame.from_records(results)
df = results_df
df

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,...,lat,lon,weapon_used_cd,weapon_desc,cross_street,vict_sex,vict_descent,crm_cd_2,crm_cd_3,crm_cd_4
0,230915611,2023-10-23T00:00:00.000,2023-10-09T00:00:00.000,1735,09,Van Nuys,0946,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,34.1794,-118.4356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,231319398,2023-10-23T00:00:00.000,2023-10-23T00:00:00.000,0920,13,Newton,1323,2,624,BATTERY - SIMPLE ASSAULT,...,34.0213,-118.2535,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",CENTRAL AV,NaN,NaN,NaN,NaN,NaN
2,230716147,2023-10-23T00:00:00.000,2023-10-22T00:00:00.000,1900,07,Wilshire,0785,1,510,VEHICLE - STOLEN,...,34.0372,-118.3423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,231516446,2023-10-23T00:00:00.000,2023-10-07T00:00:00.000,1550,15,N Hollywood,1563,1,310,BURGLARY,...,34.1522,-118.3801,NaN,NaN,NaN,F,W,NaN,NaN,NaN
4,230915602,2023-10-23T00:00:00.000,2023-10-23T00:00:00.000,0800,09,Van Nuys,0926,2,901,VIOLATION OF RESTRAINING ORDER,...,34.1877,-118.4443,NaN,NaN,NaN,F,H,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,230609985,2023-05-19T00:00:00.000,2023-05-18T00:00:00.000,2040,06,Hollywood,0632,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,34.0952,-118.3727,NaN,NaN,NaN,M,O,NaN,NaN,NaN
99996,230112664,2023-05-19T00:00:00.000,2023-05-19T00:00:00.000,2100,01,Central,0191,1,210,ROBBERY,...,34.0406,-118.268,102,HAND GUN,PICO DR,M,W,NaN,NaN,NaN
99997,230809804,2023-05-19T00:00:00.000,2023-05-09T00:00:00.000,0005,08,West LA,0829,2,354,THEFT OF IDENTITY,...,34.068,-118.4254,NaN,NaN,NaN,M,W,NaN,NaN,NaN
99998,230809844,2023-05-19T00:00:00.000,2023-05-19T00:00:00.000,1950,08,West LA,0839,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,34.0611,-118.4184,NaN,NaN,NaN,X,X,NaN,NaN,NaN


<h1> Transform Data </h1>

In [2]:
df.columns

Index(['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area', 'area_name',
       'rpt_dist_no', 'part_1_2', 'crm_cd', 'crm_cd_desc', 'mocodes',
       'vict_age', 'premis_cd', 'premis_desc', 'status', 'status_desc',
       'crm_cd_1', 'location', 'lat', 'lon', 'weapon_used_cd', 'weapon_desc',
       'cross_street', 'vict_sex', 'vict_descent', 'crm_cd_2', 'crm_cd_3',
       'crm_cd_4'],
      dtype='object')

# Process Columns Names

In [3]:
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df = df.rename(columns={'area_': 'area'})
df.columns

Index(['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area', 'area_name',
       'rpt_dist_no', 'part_1_2', 'crm_cd', 'crm_cd_desc', 'mocodes',
       'vict_age', 'premis_cd', 'premis_desc', 'status', 'status_desc',
       'crm_cd_1', 'location', 'lat', 'lon', 'weapon_used_cd', 'weapon_desc',
       'cross_street', 'vict_sex', 'vict_descent', 'crm_cd_2', 'crm_cd_3',
       'crm_cd_4'],
      dtype='object')

# Only Males and Females Values

In [4]:
df = df[(df['vict_sex'] == 'F') | (df['vict_sex'] == 'M')]

# Filter Vitcims Race

In [5]:
df = df[df['vict_descent'].notna()]
df = df[df['vict_descent'] != 'X']
df = df[df['vict_descent'] != '-']

# VICTIM AGE DATA TYPE

In [60]:
df = df.astype({'vict_age':'int'})
df = df[df['vict_age'] > 0]
df = df.astype({'lon' : 'float', 'lat' : 'float'})
df = df[df['lon'] != 0]
df = df[df['lat'] != 0]


# Drop Unnecessary Columns

In [61]:
columns_to_drop = ['mocodes', 'rpt_dist_no', 'part_1_2', 
                   'crm_cd_1', 'crm_cd_2', 'crm_cd_3', 
                   'crm_cd_4', 'cross_street']
crimes = df.drop(columns_to_drop, axis=1)
crimes.columns

Index(['dr_no', 'date_rptd', 'date_occ', 'time_occ', 'area', 'area_name',
       'crm_cd', 'crm_cd_desc', 'vict_age', 'vict_sex', 'vict_descent',
       'premis_cd', 'premis_desc', 'status', 'status_desc', 'location', 'lat',
       'lon', 'weapon_used_cd', 'weapon_desc'],
      dtype='object')

# Convert From Military Time Into Standard Time

In [62]:
crimes['time_occ'] = pd.to_datetime(crimes['time_occ'].astype(str).str.zfill(4), format='%H%M').dt.time
crimes['date_occ'] = pd.to_datetime(crimes['date_occ']).dt.date
crimes['date_rptd'] = pd.to_datetime(crimes['date_rptd']).dt.date

# Make ID Column for Each Race

In [63]:
crimes.loc[crimes['vict_descent'] == 'A', 'vict_descent'] = 'ASIAN'
crimes.loc[crimes['vict_descent'] == 'B', 'vict_descent'] = 'BLACK'
crimes.loc[crimes['vict_descent'] == 'C', 'vict_descent'] = 'CHINESE'
crimes.loc[crimes['vict_descent'] == 'D', 'vict_descent'] = 'CAMBODIAN'
crimes.loc[crimes['vict_descent'] == 'F', 'vict_descent'] = 'FILIPINO'
crimes.loc[crimes['vict_descent'] == 'G', 'vict_descent'] = 'GUAMANIAN'
crimes.loc[crimes['vict_descent'] == 'H', 'vict_descent'] = 'Hispanic/Latin/Mexican'
crimes.loc[crimes['vict_descent'] == 'I', 'vict_descent'] = 'American Indian/Alaskan Native'
crimes.loc[crimes['vict_descent'] == 'J', 'vict_descent'] = 'JAPANESE'
crimes.loc[crimes['vict_descent'] == 'K', 'vict_descent'] = 'KOREAN'
crimes.loc[crimes['vict_descent'] == 'L', 'vict_descent'] = 'LAOTIAN'
crimes.loc[crimes['vict_descent'] == 'O', 'vict_descent'] = 'OTHER'
crimes.loc[crimes['vict_descent'] == 'P', 'vict_descent'] = 'PACIFIC ISLANDER'
crimes.loc[crimes['vict_descent'] == 'S', 'vict_descent'] = 'SAMOAN'
crimes.loc[crimes['vict_descent'] == 'U', 'vict_descent'] = 'HAWAIIAN'
crimes.loc[crimes['vict_descent'] == 'V', 'vict_descent'] = 'VIETNAMESE'
crimes.loc[crimes['vict_descent'] == 'W', 'vict_descent'] = 'WHITE'
crimes.loc[crimes['vict_descent'] == 'Z', 'vict_descent'] = 'ASIAN INDIAN'
crimes['descent_id'] = pd.factorize(crimes['vict_descent'])[0]

In [64]:
crimes = crimes.drop_duplicates(subset=['dr_no'])
crimes['descent_id'].nunique()

18

# Replace NULL VALUES IN WEAPON COLUMNS WITH ACTUAL VALUE

In [65]:
crimes['weapon_used_cd'].fillna(0, inplace=True)
crimes['weapon_desc'].fillna('NO WEAPON', inplace=True)

# LAST CHECK FOR NULLS

In [66]:
crimes = crimes[crimes['premis_cd'].notna()]
crimes = crimes[crimes['premis_desc'].notna()]
crimes = crimes[crimes['status'].notna()]
crimes = crimes[crimes['area'].notna()]

# GET RID OF EXTRA SPACE IN LOCATION COLUMNS

In [67]:
crimes['location'] = crimes['location'].replace(r'\s+', ' ', regex=True)

In [68]:
crimes['location']

0                600 WALL ST
1               3000 10TH AV
3        9600 N SEPULVEDA BL
6          3900 LOS FELIZ BL
8             7400 RESEDA BL
                ...         
99993         20600 CLARK ST
99994        3500 W ADAMS BL
99995          5100 ASCOT AV
99997        8700 DE SOTO AV
99998     100 N RIDGEWOOD PL
Name: location, Length: 74734, dtype: object

# SORT AND SAVE AS CSV

In [69]:
sorted = crimes.sort_values(['date_occ', 'time_occ'])

In [70]:
sorted.isna().sum()

dr_no             0
date_rptd         0
date_occ          0
time_occ          0
area              0
area_name         0
crm_cd            0
crm_cd_desc       0
vict_age          0
vict_sex          0
vict_descent      0
premis_cd         0
premis_desc       0
status            0
status_desc       0
location          0
lat               0
lon               0
weapon_used_cd    0
weapon_desc       0
descent_id        0
dtype: int64

In [71]:
sorted.to_csv('Last_ETL.csv', index=False)

# Load Data Into DB 

In [72]:
dbEngine = create_engine(f"mysql+mysqldb://{DB}/crimes")

In [73]:
#Area
area = sorted[['area', 'area_name']]
area = area.drop_duplicates(subset=['area'])
area = area.rename(columns={'area': 'areaID', 'area_name' : 'areaName'})
area.to_sql(con = dbEngine, name = 'TEMP', if_exists='append', index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO DIM_AREA (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\1564963534.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  area.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [74]:
#LOCATION
location = sorted[['area', 'location', 'lat', 'lon']]
location = location.drop_duplicates(['location'])
location = location.rename(columns={'area': 'areaID', 'location': 'location', 'lat' : 'LAT', 'lon' : 'LON'})
location.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO DIM_LOCATION (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\2785306800.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  location.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [75]:
#PREMIS
premis = sorted[['premis_cd', 'premis_desc']]
premis = premis.drop_duplicates(subset=['premis_cd'])
premis = premis.rename(columns={'premis_cd': 'premisID', 'premis_desc': 'premisDesc'})
premis.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO DIM_PREMIS (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\3972594713.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  premis.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [76]:
#CRIMES
crimes = sorted[['crm_cd', 'crm_cd_desc']]
crimes = crimes.drop_duplicates(subset=['crm_cd'])
crimes = crimes.rename(columns={'crm_cd': 'crimeID', 'crm_cd_desc': 'crimeDesc'})
crimes.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO DIM_CRIMES (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\4066762997.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  crimes.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [77]:
#Status
status = sorted[['status', 'status_desc']]
status = status.drop_duplicates(subset=['status'])
status = status.rename(columns={'status': 'statusID', 'status_desc': 'statusDesc'})
status.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO DIM_STATUS (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\2699700056.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  status.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [78]:
#TIME
time = sorted[['dr_no', 'date_rptd', 'date_occ', 'time_occ']]
time = time.drop_duplicates(subset=['dr_no'])
time = time.rename(columns={'dr_no': 'caseNumber', 'date_rptd': 'DateReport', 'date_occ': 'DateOcc', 'time_occ': 'TimeOcc'})
time.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO DIM_TIME (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\2087642689.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  time.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [79]:
#Decent
descent = sorted[['descent_id', 'vict_descent']]
descent = descent.drop_duplicates(subset=['descent_id'])
descent = descent.rename(columns={'descent_id' : 'id', 'vict_descent': 'victDescent'})
descent.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)

with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO vict_descent (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\156934974.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  descent.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [80]:
#Victums
victums = sorted[['dr_no', 'vict_age', 'descent_id', 'vict_sex']]
victums = victums.drop_duplicates(subset=['dr_no'])
victums = victums.rename(columns={'dr_no': 'caseNumber', 'vict_age': 'VictAge', 'descent_id': 'VictDescent', 'vict_sex': 'VictSex'})
victums.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO dim_victums (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\104506362.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  victums.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [81]:
#WEAPONS
weapons = sorted[['weapon_used_cd', 'weapon_desc']]
weapons = weapons.drop_duplicates(subset=['weapon_used_cd'])
weapons = weapons.rename(columns={'weapon_used_cd' : 'weaponID', 'weapon_desc': 'weaponDesc'})
weapons.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO DIM_WEAPONS (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\3279518381.py:5: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  weapons.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)


In [82]:
#FACTS
facts = sorted[['area', 'crm_cd', 'dr_no', 'premis_cd', 'status', 'weapon_used_cd']]
facts = facts.rename(columns={'area' : 'areaID', 'crm_cd' : 'crimeID', 'dr_no' : 'caseNumber', 'premis_cd': 'premisID', 
'weapon_used_cd': 'weaponID', 'status': 'statusID'})
facts.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
with dbEngine.begin() as cnx:
    insert_sql = 'INSERT IGNORE INTO FACTS (SELECT * FROM TEMP)'
    cnx.execute(insert_sql)
    delete_sql = 'DROP TABLE TEMP'
    cnx.execute(delete_sql)

C:\Users\User\AppData\Local\Temp\ipykernel_19260\4240749436.py:4: UserWarning: The provided table name 'TEMP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  facts.to_sql(con = dbEngine, name = 'TEMP', if_exists='append',index=False)
